In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
import pickle
import requests
import numpy as np
import pandas as pd
import time
import csv
import re
import json
import xlrd
import os
from selenium.common.exceptions import NoSuchElementException
from datetime import datetime, date, timedelta
import nltk
#from flatten_json import flatten
regex = re.compile(r"\[|\]|<", re.IGNORECASE)
import time
import copy

In [2]:
%cd /home/stu/code/dans

/home/stu/code/dans


In [3]:
#file_res = "API_results_" + time.strftime("%Y%m%d") + ".csv"
file_res = "API_results_20210822.csv"
my_df = pd.read_csv(file_res)

## Categories
#my_df["Categories"]
# Willnjust keep 2 levels.
my_df["Categories"] = my_df["Categories"].map(eval, na_action='ignore')
new_df = my_df["Categories"].apply(pd.Series)
my_df["Categories"] = new_df[0].apply(pd.Series).UrlFriendlyName
my_df["Sub_Categories"] = new_df[1].apply(pd.Series).UrlFriendlyName



In [4]:

## Reviews
my_df["Reviews"] = my_df["Reviews"].map(eval,  na_action='ignore')
# Try with first 2 reviews
new_df = my_df["Reviews"].apply(pd.Series)
# First
my_df["Review1_auth"] = new_df[0].apply(pd.Series).author.apply(pd.Series).Value
my_df["Review1_authorcontent"] = new_df[0].apply(pd.Series).authorcontent.apply(pd.Series).Value
my_df["Review1_points"] = new_df[0].apply(pd.Series).points.apply(pd.Series).Value
my_df["Review1_source"] = new_df[0].apply(pd.Series).source.apply(pd.Series).Value
my_df["Review1_text"] = new_df[0].apply(pd.Series).text.apply(pd.Series).Value
my_df["Review1_vintage"] = new_df[0].apply(pd.Series).vintage.apply(pd.Series).Value
# Second
my_df["Review2_auth"] = new_df[1].apply(pd.Series).author.apply(pd.Series).Value
my_df["Review2_authorcontent"] = new_df[1].apply(pd.Series).authorcontent.apply(pd.Series).Value
my_df["Review2_points"] = new_df[1].apply(pd.Series).points.apply(pd.Series).Value
my_df["Review2_source"] = new_df[1].apply(pd.Series).source.apply(pd.Series).Value
my_df["Review2_text"] = new_df[1].apply(pd.Series).text.apply(pd.Series).Value
my_df["Review2_vintage"] = new_df[1].apply(pd.Series).vintage.apply(pd.Series).Value



In [5]:
# Additional details
my_df["AdditionalDetails"] = my_df["AdditionalDetails"].map(eval,  na_action='ignore')

In [6]:
full_df = copy.deepcopy(my_df)

In [7]:
avail = full_df[full_df["IsForDelivery"]][["Stockcode"]]

In [8]:
# Can't use nested lists of JSON objects in pd.json_normalize
my_df = my_df.explode(column="AdditionalDetails").reset_index(drop=True)

In [9]:
add_df = pd.DataFrame(pd.json_normalize(my_df["AdditionalDetails"]))

In [10]:
del add_df["DisplayName"]

In [11]:
df = pd.concat([my_df,add_df],axis=1)

In [12]:
df = df.pivot(index='Stockcode',columns='Name', values='Value')

In [13]:
df.to_csv("mydf.csv")
df = pd.read_csv("mydf.csv")

In [14]:
newdf = pd.merge(full_df, df, on='Stockcode')

In [15]:
newdf.to_csv("newdf.csv")

In [16]:
newdf = pd.read_csv("newdf.csv")

/home/stu/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (96,97,98,137,140,181) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [17]:
gives = newdf[['Stockcode','Description','webproductname','Prices.singleprice.Value','Prices.promoprice.Value','Prices.promoprice.BeforePromotion','Prices.promoprice.AfterPromotion']]

In [18]:
gives = gives[gives.webproductname.notnull()]

In [19]:
#gives = gives[gives['Stockcode'].str.contains("MYSTERY")]
gives = gives[gives["Description"].str.contains("Wraps")]

In [20]:
gives = gives[~gives["webproductname"].str.contains("Wraps")]

In [36]:
 pd.merge(gives, avail, left_on='Stockcode', right_on="Stockcode")

,Stockcode,Description,webproductname,Prices.singleprice.Value,Prices.promoprice.Value,Prices.promoprice.BeforePromotion,Prices.promoprice.AfterPromotion
0,MYSTERY437,Under Wraps Western<br>Australia Cabernet... ...,Alkoomi Blackbutt Cabernet Merlot Cabernet Franc,59.99,30.0,59.99,30.0
1,MYSTERY411,Under Wraps Hawkes Bay Syrah<br>2010 750mL,Craggy Range Le Sol Syrah 2010,155.99,79.9,155.99,79.9
2,MYSTERY459,Under Wraps Great Southern<br>Cabernet Sauvig...,Forest Hill Block 5 Caberent Sauvignon,30.00,NaN,NaN,NaN
3,MYSTERY378,Under Wraps Western<br>Australia Cabernet... ...,Peos Estate Cab Sauv,22.99,15.0,22.99,15.0
4,MYSTERY540,Under Wraps Clare Valley<br>Cabernet Sauvigno...,Jim Barry The Benbournie Cabernet Sauvignon 2014,35.88,NaN,NaN,NaN
5,MYSTERY242,Under Wraps Barossa Merlot<br>2017 750mL,Gibson My Darling Merlot 2014,24.99,14.9,24.99,14.9
6,MYSTERY594,Under Wraps Heathcote Shiraz<br>Cabernet 2017...,Feathered Friends Heathcote Shiraz Cabernet 2015,25.00,15.0,25.00,15.0
7,MYSTERY196,Under Wraps Yarra Valley<br>Chardonnay 2018 ...,Yarra Trail Yarra Valley Chardonnay 2016,31.99,15.0,31.99,15.0


In [25]:
keep =['Categories',
 'Stockcode',
 'PackageSize',
 'Prices.inanysixprice.Message',
 'Prices.inanysixprice.Value',
 'Sub_Categories',
 'Review1_auth',
 'Review1_points',
 'Review1_source',
 'awardwinner',
 'glutenfree',
 'preservativefree',
 'varietal',
 'webalcoholpercentage',
 'webbottleclosure',
 'webcountryoforigin',
 'webfoodmatch',
 'webisorganic',
 'webisvegan',
 'webliquorsize',
 'webmaincategory',
 'webregionoforigin',
 'webstateoforigin',
 'webtotalreviewcount',
 'webwinebody',
 'webwinestyle']                                                                                                      

In [38]:
keep1 =['Categories',
 'Stockcode',
 'PackageSize',
 'RichDescription',
 'Review1_text',
 'Review2_text',
 'Prices.inanysixprice.Message',
 'Prices.inanysixprice.Value',
 'Sub_Categories',
 'Review1_auth',
 'Review1_points',
 'Review1_source',
 'awardwinner',
 'glutenfree',
 'preservativefree',
 'varietal',
 'webalcoholpercentage',
 'webbottleclosure',
 'webcountryoforigin',
 'webdescriptionshort',
 'webfoodmatch',
 'webisorganic',
 'webisvegan',
 'webliquorsize',
 'webmaincategory',
 'webregionoforigin',
 'webstateoforigin',
 'webtotalreviewcount',
 'webwinebody',
 'webwinestyle']     

In [27]:




#check = pd.json_normalize(my_df["AdditionalDetails"]).pivot(columns='Name')

#check.to_csv("check.csv")

#check['Value']
#my_df["AdditionalDetails"].apply(pd.Series.explode)

########### Would likely want ot save to DB at this tome/.

#In excel: =TEXTJOIN("','",TRUE,A1:CF1)
#keep = ['Categories','BackorderMessage','varietal','DeliveryOptionsInfo','SavedLists','Stockcode','PackageSize','RichDescription','StockOnHand','UrlFriendlyName','IsDeliveryOnly','Prices.inanysixprice.Value','Prices.caseprice.Value','Prices.singleprice.Value','Inventory.availableinventoryqty','Prices.promoprice.Message','Prices.promoprice.Value','Prices.promoprice.PreText','Prices.promoprice.BeforePromotion','Prices.promoprice.AfterPromotion','Sub_Categories','Review1_auth','Review1_authorcontent','Review1_source','Review1_points','Review1_text','Review1_vintage','awardwinner','brewery','corkscoreeligible','countryoforigin','dm_stockcode','image1','image2','standarddrinks','webalcoholpercentage','webaverageproductrating','webbadgescollection','webbottleclosure','webcountryoforigin','webdescriptionshort','webdsvflag','webfoodmatch','webisvegan','weblangtonsclassification','webliquorsize','webmaincategory','webmaxquantity','webminquantity','webpacksizecase','webpacktype','webpresaleenddate','webpresaleflag','webpresalemarketlaunchdate','webpresalemdmmaxqtylimit','webpresaleorderreleasedate','webpresalestartdate','webpresaleusermaxqtylimit','webproductcanbechilled','webproductname','webproductsale','webproducttype','webpromomdmmessage','webpromomessage','webpromomessageenddt','webpromomessagestartdt','webpromosecondmesgenddt','webpromosecondmesgstartdt','webpromosecondmessage','webpromotionalbundle','webregionoforigin','webstateoforigin','webtotalreviewcount','webvideourl','webvintagecurrent','webvintagenote','webwinebody','webwinemaker','webwinestyle']
#keep = ['Categories','BackorderMessage','Description','DeliveryOptionsInfo','SavedLists','Stockcode','PackageSize','StockOnHand','UrlFriendlyName','IsDeliveryOnly','Prices.inanysixprice.Value','Prices.caseprice.Value','Prices.singleprice.Value','Inventory.availableinventoryqty','Prices.promoprice.Message','Prices.promoprice.Value','Prices.promoprice.PreText','Prices.promoprice.BeforePromotion','Prices.promoprice.AfterPromotion','Sub_Categories','Review1_auth','Review1_authorcontent','Review1_source','Review1_points','Review1_text','Review1_vintage','awardwinner','brewery','corkscoreeligible','countryoforigin','dm_stockcode','image1','image2','standarddrinks','webalcoholpercentage','webaverageproductrating','webbadgescollection','webbottleclosure','webcountryoforigin','webdescriptionshort','webdsvflag','webfoodmatch','webisvegan','weblangtonsclassification','webliquorsize','webmaincategory','webmaxquantity','webminquantity','webpacksizecase','webpacktype','webpresaleenddate','webpresaleflag','webpresalemarketlaunchdate','webpresalemdmmaxqtylimit','webpresaleorderreleasedate','webpresalestartdate','webpresaleusermaxqtylimit','webproductcanbechilled','webproductname','webproductsale','webproducttype','webpromomdmmessage','webpromomessage','webpromomessageenddt','webpromomessagestartdt','webpromosecondmesgenddt','webpromosecondmesgstartdt','webpromosecondmessage','webpromotionalbundle','webregionoforigin','webstateoforigin','webtotalreviewcount','webvideourl','webvintagecurrent','webvintagenote','webwinebody','webwinemaker','webwinestyle']
# Timmed version
#### OLD KEEP
#keep = ['Description','Stockcode','PackageSize','Prices.inanysixprice.Value','Prices.caseprice.Value','Prices.singleprice.Value','Prices.promoprice.Message','Prices.promoprice.Value','Prices.promoprice.PreText','Sub_Categories','Review1_auth','Review1_points','Review1_vintage','awardwinner','countryoforigin','standarddrinks','webalcoholpercentage','webaverageproductrating','webbadgescollection','webbottleclosure','webcountryoforigin','webdescriptionshort','webdsvflag','webfoodmatch','webisvegan','weblangtonsclassification','webliquorsize','webmaincategory','webpacksizecase','webpacktype','webregionoforigin','webstateoforigin','webvintagecurrent','webvintagenote','webwinebody','webwinestyle']
# Now Keep them.
kept = newdf[keep]
#kept.to_csv('fri_big_run_kept.csv')
kept.reset_index(drop=True, inplace=True)

In [107]:
kept.to_csv("kept.csv")

In [108]:
#text to TDIF
#'webdescriptionshort',
#'RichDescription',

In [109]:
#kept = pd.read_csv("kept.csv")

In [110]:
known = kept
known.reset_index(drop=True, inplace=True)

In [111]:
#### ONE HOT ENCODED
##### First I split into numeric and nominal. OHE the nominal
exclude_col = known.select_dtypes(include=np.number).columns.tolist() + ["Stockcode"]
my_df_num = known[exclude_col]
my_df_cat = known.drop(exclude_col, axis=1)
#my_df_cat.to_csv("FIN.csv")
my_df_cat_ohe = pd.get_dummies(my_df_cat)
my_df_ohe = pd.concat([my_df_num,my_df_cat_ohe], axis=1)
my_df_ohe = my_df_ohe.fillna(0)
my_df_ohe = my_df_ohe.replace(np.nan, 0)

# Drop duplicates #TODO check whats better to keep
my_df_ohe = my_df_ohe.loc[:,~my_df_ohe.columns.duplicated()]
# Clean up names
my_df_ohe.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in my_df_ohe.columns.values]

myst  = my_df_ohe[my_df_ohe.Stockcode.str.contains("MYSTERY")]
known  = my_df_ohe[~my_df_ohe.Stockcode.str.contains("MYSTERY")]

myst_nn = myst.drop("Stockcode", axis=1)
known_nn = known.drop("Stockcode", axis=1)


In [112]:

from sklearn.neighbors import NearestNeighbors
# Create the k-NN model using k=5
nn_abs = NearestNeighbors(n_neighbors=1, algorithm='auto')

# Fit it
nn_abs.fit(known_nn)

#distance, matches = nn_abs.kneighbors(myst_nn.iloc[[9]], 2, return_distance=True)
#matches

#known["Stockcode"].iloc[matches[0]]

#myst['Stockcode'].iloc[[9]]
#Now, tokenise each descition.
#MAGIC TIME

#myst.iloc[[9]].to_csv("myst.csv")
#myst_nn.iloc[[9]].to_csv("myst_nn.csv")



#known.iloc[matches[0]].to_csv("known.csv")
#known_nn.iloc[matches[0]].to_csv("known_nn.csv")

results_wine = []

for index in range(len(myst.index)):
    distance, matches = nn_abs.kneighbors(myst_nn.iloc[[index]], 1, return_distance=True)
    results_wine.append(
        {
            'Mystery': "https://www.danmurphys.com.au/product/" + str(myst['Stockcode'].iloc[[index][0]]),
            'Mystery Stockcode': str(myst['Stockcode'].iloc[[index][0]]),
            'Matched': "https://www.danmurphys.com.au/product/" + str(known["Stockcode"].iloc[matches[0][0]]),
            'Matched Stockcode':  str(known["Stockcode"].iloc[matches[0][0]]),
            'Distance': str(distance[0][0])
            
        }
    )


matched = pd.DataFrame(results_wine)

#def create_clickable_id(id):
#    url_template= '''<a href="../../link/to/{id}" target="_blank">{id}</a>'''.format(id=id)
#    return url_template

#matched['Mystery'] = matched['Mystery'].apply(create_clickable_id)
#matched['Matched'] = matched['Matched'].apply(create_clickable_id)

#file_match = "matched_results_" + time.strftime("%Y%m%d") + ".csv"

#matched.to_html("matched.html")



#matched.to_csv("matched_sun_1.csv")

#matches = nn_abs.kneighbors(myst_nn.iloc[[3]], 2, return_distance=False)
#known["Stockcode"].iloc[matches[0]]

#myst['Stockcode'].iloc[[3]]




In [113]:
#matched.to_csv("Sat_pm.csv")

In [114]:
#TODO Now just need to join back with the stockcode, and we've got a working MVP..
promos = newdf[newdf.Stockcode.str.contains("MYSTERY")][['Stockcode','webproductname','Prices.singleprice.Value','Prices.promoprice.Value']]
cutdown = newdf[['Stockcode','Description','producttitle']]

In [115]:
final = pd.merge(matched, promos, left_on='Mystery Stockcode', right_on='Stockcode')

In [116]:
final = pd.merge(final, cutdown, left_on='Matched Stockcode', right_on='Stockcode')

In [117]:
final = pd.merge(final, avail, left_on='Mystery Stockcode', right_on="Stockcode")

In [118]:
final = final[["Mystery","Matched","Distance","webproductname","Prices.singleprice.Value","Prices.promoprice.Value","Description","producttitle"]]

In [119]:
final['Savings'] = final["Prices.singleprice.Value"] - final["Prices.promoprice.Value"]

In [120]:
final["MatchLevel"] = np.where(final['Distance'].astype(float) < 1.8, "Good", "Poor")
final = final.sort_values(['MatchLevel', 'Savings'], ascending=[True, False])

In [121]:
final = final[final.MatchLevel.str.contains("Good")]

In [53]:
def make_clickable(val):
    # target _blank to open new window
    return '<a target="_blank" href="{}">{}</a>'.format(val, val)

final = final.style.format({'Mystery': make_clickable,'Matched': make_clickable,})\
                   .bar(subset=['Savings'], align='mid', color=['#5fba7d'])\
                   .bar(subset=['Savings'], align='mid', color=['#5fba7d'])\
                   .hide_index()\
    
        

In [54]:

#final = final.style.format({'Mystery': make_clickable,'Matched': make_clickable,})
#final = final.style.format({'Matched': make_clickable})

In [55]:
#writing HTML Content
heading = '<h1> Matched wines</h1>'
subheading = '<h3> Results sub header </h3>'
# Using .now() from datetime library to add Time stamp
now = datetime.now()
current_time = now.strftime("%m/%d/%Y %H:%M:%S")
header = '<div class="top">' + heading + subheading +'</div>'
footer = '<div class="bottom"> <h3> This Report has been Generated on'+ current_time +'</h3> </div>'
content = final
# Concating everything to a single string

In [56]:
html = header + content.render() + footer
# Writing the file
with open('report.html','w+') as file:
    file.write(html)
    #file.write(content)
    #file.write(footer)

In [57]:
#final.to_html("final")

AttributeError: 'Styler' object has no attribute 'to_html'

In [101]:
final_res = "Matched_results_" + time.strftime("%Y%m%d") + ".html"
f=open(final_res,"w")
f.write(final.render()) # df is the styled dataframe
f.close()
#final.to_html(final_res)

# Now any easy ones just matching descirptions

In [58]:
kept = newdf[keep1]
#kept.to_csv('fri_big_run_kept.csv')
newdf['Review1_text'] = full_df['Review1_text']
newdf['Review2_text'] = full_df['Review2_text']

kept.reset_index(drop=True, inplace=True)
myst  = kept[kept.Stockcode.str.contains("MYSTERY")]
known  = kept[~kept.Stockcode.str.contains("MYSTERY")]


In [59]:
desc_match = pd.merge(myst[myst['RichDescription'].notna()], known, on=['RichDescription'], how='inner')

In [60]:
webdesc_match = pd.merge(myst[myst['webdescriptionshort'].notna()], known, on=['webdescriptionshort'], how='inner')

In [61]:
rev_match = pd.merge(myst[myst['Review1_text'].notna()], known, on=['Review1_text'], how='inner')

In [62]:
rev1_match = pd.merge(myst[myst['Review2_text'].notna()], known, on=['Review2_text'], how='inner')

In [63]:
text_match = desc_match.append(webdesc_match).append(rev_match).append(rev1_match).reset_index()

In [64]:
text_match = text_match[['Stockcode_x','Stockcode_y']]

In [65]:
#TODO Now just need to join back with the stockcode, and we've got a working MVP..
#promos = newdf[newdf.Stockcode.str.contains("MYSTERY")][['Stockcode','webproductname','Prices.singleprice.Value','Prices.promoprice.Value']]
#cutdown = newdf[['Stockcode','Description','producttitle']]

In [66]:
text_match = pd.merge(text_match, newdf, left_on='Stockcode_x', right_on='Stockcode')[['Stockcode_x','Stockcode_y','Description', 'Prices.singleprice.Value','Prices.promoprice.Value']]

In [67]:
text_match = pd.merge(text_match, newdf, left_on='Stockcode_y', right_on='Stockcode')[['Stockcode_x','Stockcode_y','Description_y', 'Prices.singleprice.Value_x','Prices.promoprice.Value_x']]


In [68]:
text_match['Savings'] = text_match["Prices.singleprice.Value_x"] - text_match["Prices.promoprice.Value_x"]

In [69]:
text_match = text_match.sort_values(['Savings'], ascending=[False])

In [70]:
text_match.to_csv("Text_match")

In [ ]:
#Try Fuzzy
#https://towardsdatascience.com/how-to-do-fuzzy-matching-in-python-pandas-dataframe-6ce3025834a6

# Now to try and do the NLP Work!

In [29]:
#Text to TFIDF
#'webdescriptionshort',
#'RichDescription',
keep1 =['Stockcode',
 'webdescriptionshort',
'RichDescription']     
#keep1 =[
# 'webdescriptionshort',
#'RichDescription','Review1_text','Review2_text']     


In [30]:
kept = newdf[keep1]
#kept.to_csv('fri_big_run_kept.csv')
kept.reset_index(drop=True, inplace=True)

In [59]:
myst_nn  = kept[kept.Stockcode.str.contains("MYSTERY")]
known_nn  = kept[~kept.Stockcode.str.contains("MYSTERY")]

In [73]:
# Using http://mlbernauer.github.io/R/20160131-document-retrieval-sklearn.html

In [60]:
## Adding in tfidf for description, and then dropping the columns

from sklearn.feature_extraction.text import TfidfVectorizer

#transformer = TfidfVectorizer(max_features=1000, stop_words='english')

#v = TfidfVectorizer(ngram_range=(2, 3))
#x = v.fit_transform(kept['RichDescription'].values.astype('U'))



In [61]:
# Create vectorizer for webdescriptionshort, max_df is set to 0.5, we only want
# to include terms that appear in less tha 50% of the documents (i.e. rare terms)
web_tfidf_vectorizer = TfidfVectorizer(max_features=1000000, use_idf=True,ngram_range=(1, 3))

# Create vectorizer for , RichDescriptions max_df is set to 0.5, we only want 
# to include terms that appear in less than 50% of the documents (i.e. rare terms)
rich_tfidf_vectorizer = TfidfVectorizer(max_features=1000000, use_idf=True,ngram_range=(1, 3))

In [62]:
# Fit Weights for both
tfidf_weights_web = web_tfidf_vectorizer.fit_transform(kept['webdescriptionshort'].values.astype('U'))
tfidf_weights_rich = rich_tfidf_vectorizer.fit_transform(kept['RichDescription'].values.astype('U'))


In [63]:
# Get feature names for Abstract and Title models
tfidf_features_web = web_tfidf_vectorizer.get_feature_names()
tfidf_features_rich = rich_tfidf_vectorizer.get_feature_names()

In [64]:
# Build model to return 5 closest neighbors
from sklearn.neighbors import NearestNeighbors

# Create the k-NN model using k=5
nn_web = NearestNeighbors(n_neighbors=1, algorithm='auto')
nn_rich = NearestNeighbors(n_neighbors=1, algorithm='auto')

# Fit the models to the TF-IDF weights matrix
nn_fitted_web = nn_web.fit(tfidf_weights_web)
nn_fitted_rich = nn_rich.fit(tfidf_weights_rich)

In [65]:
def find_nearest_papers(row, kNNmodel, tfidf_weights, tfidf_features, papers):
    #keywords = get_top_features(row, tfidf_weights, tfidf_features)
    dist,idx = kNNmodel.kneighbors(tfidf_weights[row,:])
    idx = list(idx[0])
    return {'Wines':kept.iloc[idx]}


In [66]:
def get_top_features(rownum, weights, features, top_k=10):
    weight_vec = weights.toarray()[rownum,:]
    top_idx = np.argsort(weight_vec)[::-1][:top_k]
    return [features[i] for i in top_idx]

In [37]:
kept.index[kept['Stockcode'] == 'MYSTERY265']

Int64Index([1115], dtype='int64')

In [47]:
kept.index[kept['Stockcode'] == 'MYSTERY265']

Int64Index([1115], dtype='int64')

In [43]:
#df.index[df['Stockcode'] == 'MYSTERY265']

Int64Index([5354], dtype='int64')

In [48]:
#find_nearest_papers(1, nn_fitted_abs, tfidf_weights_abs, tfidf_features_abs, papers)['papers']
find_nearest_papers(1115, nn_fitted_web, tfidf_weights_web, tfidf_features_web, kept)['Wines']

,Stockcode,webdescriptionshort,RichDescription
1115,MYSTERY265,<p>The fruit for this iconic wine is handpicke...,NaN


In [49]:
#find_nearest_papers(1, nn_fitted_abs, tfidf_weights_abs, tfidf_features_abs, papers)['papers']
find_nearest_papers(1115, nn_fitted_rich, tfidf_weights_rich, tfidf_features_rich, kept)['Wines']

,Stockcode,webdescriptionshort,RichDescription
54,943434,"Dark and focused, filled with juicy and vibran...",NaN


In [ ]:
results_wine_nlp = []

for index in range(len(myst.index)):
    distance, matches = nn_abs.kneighbors(myst_nn.iloc[[index]], 1, return_distance=True)
    results_wine_nlp.append(
        {
            'Mystery': "https://www.danmurphys.com.au/product/" + str(myst['Stockcode'].iloc[[index][0]]),
            'Mystery Stockcode': str(myst['Stockcode'].iloc[[index][0]]),
            'Matched': "https://www.danmurphys.com.au/product/" + str(known["Stockcode"].iloc[matches[0][0]]),
            'Matched Stockcode':  str(known["Stockcode"].iloc[matches[0][0]]),
            'Distance': str(distance[0][0])
            
        }
    )


matched = pd.DataFrame(results_wine)



In [86]:

#my_df_ohe.to_csv("OHE.csv")
#myst_nn = myst_nn.reset_index()
#myst = myst.reset_index()

#known_nn = known_nn.reset_index()
#known = known.reset_index()

##### Now myst
#my_df_num_m = myst[exclude_col]
#my_df_cat_m = myst.drop(exclude_col, axis=1)
#my_df_cat.to_csv("FIN.csv")
#my_df_cat_ohe_m = pd.get_dummies(my_df_cat_m)
#my_df_ohe_m = pd.concat([my_df_num_m,my_df_cat_ohe_m], axis=1)
#my_df_ohe_m = my_df_ohe_m.drop("Stockcode", axis=1)
#my_df_ohe_m = my_df_ohe_m.fillna(0)
#my_df_ohe_m = my_df_ohe_m.replace(np.nan, 0)


from sklearn.neighbors import NearestNeighbors
# Create the k-NN model using k=5
nn_abs = NearestNeighbors(n_neighbors=5, algorithm='auto')


# Checkif we've got NsNS

#
#wines_corr =  my_df_ohe_d.corr(method = "pearson")
#
nn_abs.fit(known_nn)

distance, matches = nn_abs.kneighbors(myst_nn.iloc[[9]], 2, return_distance=True)
#matches

known["Stockcode"].iloc[matches[0]]

myst['Stockcode'].iloc[[9]]
#Now, tokenise each descition.
#MAGIC TIME

myst.iloc[[9]].to_csv("myst.csv")
myst_nn.iloc[[9]].to_csv("myst_nn.csv")



known.iloc[matches[0]].to_csv("known.csv")
known_nn.iloc[matches[0]].to_csv("known_nn.csv")

results_wine = []

for index in range(len(myst.index)):
    distance, matches = nn_abs.kneighbors(myst_nn.iloc[[index]], 1, return_distance=True)
    results_wine.append(
        {
            'Mystery': "https://www.danmurphys.com.au/product/" + str(myst['Stockcode'].iloc[[index][0]]),
            'Matched': "https://www.danmurphys.com.au/product/" + str(known["Stockcode"].iloc[matches[0][0]]),
            'Distance': str(distance[0][0])
        }
    )


matched = pd.DataFrame(results_wine)

#def create_clickable_id(id):
#    url_template= '''<a href="../../link/to/{id}" target="_blank">{id}</a>'''.format(id=id)
#    return url_template

#matched['Mystery'] = matched['Mystery'].apply(create_clickable_id)
#matched['Matched'] = matched['Matched'].apply(create_clickable_id)

matched.to_html("matched.html")

matched.to_csv("matched_sat_1.csv")

matches = nn_abs.kneighbors(myst_nn.iloc[[3]], 2, return_distance=False)
known["Stockcode"].iloc[matches[0]]

myst['Stockcode'].iloc[[3]]




https://www.danmurphys.com.au/product/DM_MYSTERY352/under-wraps-beechworth-sangiovese-2018

https://www.danmurphys.com.au/product/7252


## xgboost

from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X = known_nn
Y = known['Stockcode']

X_test = myst_nn
Y_test = myst['Stockcode']

# Dont need this if already cleaned ohe
#X.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in X.columns.values]


#
seed = 7
test_size = 0.33
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)


# fit model no training data
model = XGBClassifier(verbosity=2)
model.fit(X, Y)

# Check it out now
print(model)


### NB
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
clf.fit(X, Y)
GaussianNB(priors=None)
y_pred = clf.predict(X_test_std)
## Now to predict

y_pred = clf.predict(X_test)
#y_pred holds the predicted label of your test set.
## Finally time to see the accuracy of our estimator.

from sklearn.metrics import accuracy_score
accuracy_score(y_true=y_test, y_pred=y_pred)

pd.DataFrame(y_pred).to_csv("Pred_NB.CSV")

pd.DataFrame(Y_test).to_csv("Myst_NB.CSV")Ansd t

SyntaxError: invalid syntax (<ipython-input-86-9f0de12e65a0>, line 82)